# Ecohome Sandbox Notebook

## Weather Forecast

In [38]:
# Load Library and URL Key
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
llm_base_url = "https://openai.vocareum.com/v1"
# weather_api_url = "https://geocoding-api.open-meteo.com/v1/search"
weather_api_url = "https://api.open-meteo.com/v1/forecast"
# weather_api_url = "https://api.openweathermap.org/data/3.0/onecall"

In [39]:
# Function Definitions

def get_weather_forecast(location: str, days: int = 3) -> Dict[str, Any]:
    """
    Get weather forecast for a specific location and number of days.
    
    Args:
        location (str): Location to get weather for (e.g., "San Francisco, CA")
        days (int): Number of days to forecast (1-7)
    
    Returns:
        Dict[str, Any]: Weather forecast data including temperature, conditions, and solar irradiance
        E.g:
        forecast = {
            "location": ...,
            "forecast_days": ...,
            "current": {
                "temperature_c": ...,
                "condition": random.choice(["sunny", "partly_cloudy", "cloudy"]),
                "humidity": ...,
                "wind_speed": ...
            },
            "hourly": [
                {
                    "hour": ..., # for hour in range(24)
                    "temperature_c": ...,
                    "condition": ...,
                    "solar_irradiance": ...,
                    "humidity": ...,
                    "wind_speed": ...
                },
            ]
        }
    """
    # --- Input Validation ---
    if not isinstance(days, int) or not (1 <= days <= 7):
        print("Error: 'days' must be an integer between 1 and 7.")
        return None

    # --- Step 1: Geocoding (Get Lat/Lon) using Nominatim ---
    geolocator = Nominatim(user_agent="weather_forecast_app")
    lat, lon, resolved_city_name = None, None, None

    try:
        # Use a timeout to prevent hanging
        geoinfo = geolocator.geocode(location, timeout=10)
        
        if geoinfo:
            lat = geoinfo.latitude
            lon = geoinfo.longitude
            # Use the full address returned by Nominatim as the resolved name
            # resolved_city_name = geoinfo.address
            resolved_city_name = geoinfo.raw["name"]

            print(f"The location is: {resolved_city_name}")
            print(f"Latitude: {lat}")
            print(f"Longitude: {lon}")
        else:
            print(f"Could not find coordinates for '{location}' using Nominatim.")
            return None # Exit immediately if geocoding fails

    except Exception as e:
        print(f"Geocoding Error: {e}")
        return None

    # Check for successful geocoding before proceeding
    if lat is None or lon is None:
        print("Error: Geocoding failed to find valid coordinates.")
        return None

    # --- Step 2: Fetch Weather Data ---
    weather_url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": lat,
        "longitude": lon,
        "forecast_days": days,
        "current": ["temperature_2m", "weather_code", "relative_humidity_2m", "wind_speed_10m"],
        "hourly": ["temperature_2m", "weather_code", "relative_humidity_2m", "wind_speed_10m", "shortwave_radiation"],
        "timezone": "auto"
    }

    try:
        w_response = requests.get(weather_url, params=weather_params)
        w_response.raise_for_status()
        data = w_response.json()

        current = data.get("current", {})
        hourly = data.get("hourly", {})

        # --- Step 3: Build Output Dictionary ---
        forecast = {
            "location": resolved_city_name,
            "forecast_days": days,
            "current": {
                "temperature_c": current.get("temperature_2m"),
                "condition": get_condition_string(current.get("weather_code")),
                "humidity": current.get("relative_humidity_2m"),
                "wind_speed": current.get("wind_speed_10m")
            },
            "hourly": []
        }

        # Loop through the hourly data lists
        total_hours = len(hourly.get("time", []))
        
        for i in range(total_hours):
            # datetime.fromisoformat handles the ISO 8601 string format
            dt = datetime.fromisoformat(hourly["time"][i])
            
            hourly_entry = {
                "hour": dt.hour,
                "temperature_c": hourly["temperature_2m"][i],
                "condition": _get_condition_string(hourly["weather_code"][i]),
                "solar_irradiance": hourly["shortwave_radiation"][i],
                "humidity": hourly["relative_humidity_2m"][i],
                "wind_speed": hourly["wind_speed_10m"][i]
            }
            forecast["hourly"].append(hourly_entry)

        return forecast

    except requests.exceptions.RequestException as e:
        print(f"Network/HTTP Error during Weather API call: {e}")
        return None
    except Exception as e:
        print(f"Weather API processing error: {e}")
        return None

def _get_condition_string(wmo_code: int) -> str:
    """Maps WMO weather codes to specific strings."""
    # 0: Clear sky
    if wmo_code == 0:
        return "sunny"
    # 1, 2, 3: Mainly clear, partly cloudy, and overcast
    elif wmo_code in [1, 2, 3]:
        return "partly_cloudy"
    # All other codes (Fog, Rain, Snow, Thunderstorm)
    else:
        return "cloudy"

In [40]:
# Testing

location = "San Francisco, CA"
days = 5
get_weather_forecast(location, days)

The location is: San Francisco
Latitude: 37.7792588
Longitude: -122.4193286


{'location': 'San Francisco',
 'forecast_days': 5,
 'current': {'temperature_c': 13.8,
  'condition': 'sunny',
  'humidity': 80,
  'wind_speed': 17.7},
 'hourly': [{'hour': 0,
   'temperature_c': 9.0,
   'condition': 'sunny',
   'solar_irradiance': 0.0,
   'humidity': 84,
   'wind_speed': 15.8},
  {'hour': 1,
   'temperature_c': 8.9,
   'condition': 'partly_cloudy',
   'solar_irradiance': 0.0,
   'humidity': 87,
   'wind_speed': 16.2},
  {'hour': 2,
   'temperature_c': 8.9,
   'condition': 'sunny',
   'solar_irradiance': 0.0,
   'humidity': 87,
   'wind_speed': 15.7},
  {'hour': 3,
   'temperature_c': 9.2,
   'condition': 'partly_cloudy',
   'solar_irradiance': 0.0,
   'humidity': 88,
   'wind_speed': 16.3},
  {'hour': 4,
   'temperature_c': 9.0,
   'condition': 'sunny',
   'solar_irradiance': 0.0,
   'humidity': 87,
   'wind_speed': 20.2},
  {'hour': 5,
   'temperature_c': 8.8,
   'condition': 'sunny',
   'solar_irradiance': 0.0,
   'humidity': 87,
   'wind_speed': 21.9},
  {'hour': 6

## Electricity Cost

In [43]:
# Function Definitions
def get_electricity_prices(date: str = None) -> Dict[str, Any]:
    """
    Get electricity prices for a specific date or current day.
    
    Args:
        date (str): Date in YYYY-MM-DD format (defaults to today)
    
    Returns:
        Dict[str, Any]: Electricity pricing data with hourly rates 
        E.g: 
        prices = {
            "date": ...,
            "pricing_type": "time_of_use",
            "currency": "USD",
            "unit": "per_kWh",
            "hourly_rates": [
                {
                    "hour": .., # for hour in range(24)
                    "rate": ..,
                    "period": ..,
                    "demand_charge": ...
                }
            ]
        }
    """
    if date is None:
        date = datetime.now().strftime("%Y-%m-%d")
    
    # Mock electricity pricing - in real implementation, this would call a pricing API
    # Use a base price per kWh    
    # Then generate hourly rates with peak/off-peak pricing
    # Peak normally between 6 and 22...
    # demand_charge should be 0 if off-peak
    
    # --- Time-of-Use Rate Simulation Parameters (Realistic Example Rates) ---
    BASE_RATE_CENTS = 12.0  # $0.12 per kWh baseline
    DEMAND_CHARGE_RATE = 8.0 # $8.00 per kW (for peak period only)

    # Define the Time-of-Use (TOU) periods
    # Note: These are typical summer/daylight hours.
    RATES = {
        "OFF_PEAK": {"multiplier": 0.8, "period": "Off-Peak", "demand_charge": 0.0}, # Lowest rate
        "MID_PEAK": {"multiplier": 1.1, "period": "Mid-Peak", "demand_charge": 0.0}, # Moderate rate
        "PEAK":     {"multiplier": 1.6, "period": "Peak", "demand_charge": DEMAND_CHARGE_RATE}, # Highest rate and includes demand charge
    }

    hourly_rates: List[Dict[str, Any]] = []

    for hour in range(24):
        rate_info = RATES["MID_PEAK"]  # Default to mid-peak

        if 23 <= hour or hour < 6:  # 11 PM to 5 AM (7 hours)
            rate_info = RATES["OFF_PEAK"]
        elif 14 <= hour < 20: # 2 PM to 7 PM (6 hours) - High demand time
            rate_info = RATES["PEAK"]
        # All other hours remain MID_PEAK (6 AM - 2 PM, 8 PM - 11 PM)

        # Calculate the actual rate
        rate_per_kwh = (BASE_RATE_CENTS * rate_info["multiplier"]) / 100.0
        hourly_rates.append({
            "hour": hour,
            "rate": round(rate_per_kwh, 4), # Rate in USD per kWh
            "period": rate_info["period"],
            "demand_charge": rate_info["demand_charge"] 
        })
    
    # Construct the final output dictionary
    prices = {
        "date": date,
        "pricing_type": "time_of_use",
        "region": "Simulated US Utility", # Specify region, as real data is region-dependent
        "currency": "USD",
        "unit": "per_kWh",
        "demand_unit": "per_kW",
        "hourly_rates": hourly_rates
    }
    return prices

In [ ]:
# Testing
get_electricity_prices("2025-11-25")

{'date': '2025-11-25',
 'pricing_type': 'time_of_use',
 'region': 'Simulated US Utility',
 'currency': 'USD',
 'unit': 'per_kWh',
 'demand_unit': 'per_kW',
 'hourly_rates': [{'hour': 0,
   'rate': 0.096,
   'period': 'Off-Peak',
   'demand_charge': 0.0},
  {'hour': 1, 'rate': 0.096, 'period': 'Off-Peak', 'demand_charge': 0.0},
  {'hour': 2, 'rate': 0.096, 'period': 'Off-Peak', 'demand_charge': 0.0},
  {'hour': 3, 'rate': 0.096, 'period': 'Off-Peak', 'demand_charge': 0.0},
  {'hour': 4, 'rate': 0.096, 'period': 'Off-Peak', 'demand_charge': 0.0},
  {'hour': 5, 'rate': 0.096, 'period': 'Off-Peak', 'demand_charge': 0.0},
  {'hour': 6, 'rate': 0.132, 'period': 'Mid-Peak', 'demand_charge': 0.0},
  {'hour': 7, 'rate': 0.132, 'period': 'Mid-Peak', 'demand_charge': 0.0},
  {'hour': 8, 'rate': 0.132, 'period': 'Mid-Peak', 'demand_charge': 0.0},
  {'hour': 9, 'rate': 0.132, 'period': 'Mid-Peak', 'demand_charge': 0.0},
  {'hour': 10, 'rate': 0.132, 'period': 'Mid-Peak', 'demand_charge': 0.0},
  {'